In [58]:

import numpy as np
#from keras_tuner.tuners import RandomSearch, BayesianOptimization, Hyperband
import tensorflow as tf
from kerastuner import HyperParameter, HyperParameters
from kerastuner.tuners import RandomSearch, BayesianOptimization, Hyperband


In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
# Normalize the pixel values to the range of [0, 1].
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Add the channel dimension to the images.
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

In [4]:
def build_model(hp):
    hp_conv_layers = hp.Int("conv_layers", 1, 3, default=3)
    #hp_dropout= hp.Boolean("dropout")
    hp_dropout_rate=hp.Choice("dropout_rate", values=[0.0, 0.1, 0.2, 0.3, 0.4, 0.5]) 
    hp_activation=hp.Choice('activation', values=['relu', 'tanh'])
    hp_optimizer = hp.Choice('optimizer', values=['adam', 'SGD', 'rmsprop'])
    hp_learningrate=hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])
    inputs = tf.keras.Input(shape=(28, 28, 1))
    x=inputs
   
    for i in range(hp_conv_layers):
        
        x = tf.keras.layers.Conv2D(
            filters=hp.Int("filters_" + str(i), 4, 32, step=4, default=8),
            kernel_size=hp.Int("kernel_size_" + str(i), 3, 5),
            activation=hp_activation,
            padding="same",
        )(x)

    x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = tf.keras.layers.Flatten()(x)

    # A hyperparamter for whether to use dropout layer.
    #if hp.Boolean("dropout"):
        #print(hp_dropout)
    x = tf.keras.layers.Dropout(hp_dropout_rate)(x)
    
    # The last layer contains 10 units,
    # which is the same as the number of classes.
    outputs = tf.keras.layers.Dense(units=10, activation="softmax")(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    '''if hp.Choice("optimizer", ["adam", "sgd", "rmsprop"]) == "adam":
        hp_optimizer=tf.keras.optimizers.Adam(hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4]), beta_1=0.9, 
                                              beta_2=0.999, epsilon=1e-07)
    elif hp.Choice("optimizer", ["adam", "sgd", "rmsprop"]) == "sgd":
        hp_optimizer=tf.keras.optimizers.SGD(hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4]),momentum=0.0)
    elif hp.Choice("optimizer", ["adam", "sgd", "rmsprop"]) == "rmsprop":
        hp_optimizer=tf.keras.optimizers.RMSprop(hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4]),rho=0.9)
    '''
    model.compile(
        loss="sparse_categorical_crossentropy", metrics=["accuracy"], optimizer=hp_optimizer,
    )
    return model


In [5]:
# Initialize the `HyperParameters` and set the values.
hp = HyperParameters()

In [6]:
model = build_model(hp)
# Test if the model runs with our data.
#model(x_train[:100])
# Print a summary of the model.
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 8)         80        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 8)         584       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 8)         584       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 8)         0         
_________________________________________________________________
flatten (Flatten)            (None, 1568)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1568)             

In [7]:
tuner = RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=15,
    executions_per_trial=2,
    overwrite=True,
    directory=".",
    project_name="keras_trial",
)

In [8]:
tuner.search(x_train, y_train, epochs=2, validation_data=(x_test, y_test))

Trial 15 Complete [00h 00m 16s]
val_accuracy: 0.9461999833583832

Best val_accuracy So Far: 0.9877499938011169
Total elapsed time: 00h 05m 26s
INFO:tensorflow:Oracle triggered exit


In [9]:
best_hps = tuner.get_best_hyperparameters()[0]
print(best_hps.values)

best_model = tuner.get_best_models()[0]

{'conv_layers': 3, 'dropout_rate': 0.1, 'activation': 'relu', 'optimizer': 'rmsprop', 'learning_rate': 0.01, 'filters_0': 20, 'kernel_size_0': 5, 'filters_1': 8, 'kernel_size_1': 5, 'filters_2': 12, 'kernel_size_2': 5}


In [10]:
best_model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 20)        520       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 8)         4008      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 12)        2412      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 12)        0         
_________________________________________________________________
flatten (Flatten)            (None, 2352)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2352)             

In [13]:
tuner.results_summary()

Results summary
Results in .\keras_trial
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
conv_layers: 3
dropout_rate: 0.1
activation: relu
optimizer: rmsprop
learning_rate: 0.01
filters_0: 20
kernel_size_0: 5
filters_1: 8
kernel_size_1: 5
filters_2: 12
kernel_size_2: 5
Score: 0.9877499938011169
Trial summary
Hyperparameters:
conv_layers: 2
dropout_rate: 0.4
activation: relu
optimizer: rmsprop
learning_rate: 0.0001
filters_0: 8
kernel_size_0: 4
filters_1: 20
kernel_size_1: 4
filters_2: 20
kernel_size_2: 4
Score: 0.9828000068664551
Trial summary
Hyperparameters:
conv_layers: 2
dropout_rate: 0.1
activation: tanh
optimizer: rmsprop
learning_rate: 0.001
filters_0: 20
kernel_size_0: 3
filters_1: 20
kernel_size_1: 3
filters_2: 28
kernel_size_2: 4
Score: 0.9802000224590302
Trial summary
Hyperparameters:
conv_layers: 2
dropout_rate: 0.1
activation: tanh
optimizer: adam
learning_rate: 0.0001
filters_0: 12
kernel_size_0: 3
filters_1: 20
kernel

In [55]:
hyperband_tuner = Hyperband(
    hypermodel=build_model,
    objective="val_accuracy",
    max_epochs=2,
    factor=3,
    hyperband_iterations=15,
    #distribution_strategy=tf.distribute.MirroredStrategy(),
    directory=".r",
    project_name="keras_trial",
    overwrite=True,
)

In [56]:
hyperband_tuner.search(x_train, y_train, epochs=2, validation_data=(x_test, y_test))

Trial 30 Complete [00h 00m 09s]
val_accuracy: 0.9544000029563904

Best val_accuracy So Far: 0.9886000156402588
Total elapsed time: 00h 05m 43s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [57]:
hyperband_tuner.results_summary()

Results summary
Results in .r\keras_trial
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
conv_layers: 3
dropout_rate: 0.3
activation: relu
optimizer: adam
learning_rate: 0.001
filters_0: 20
kernel_size_0: 3
filters_1: 24
kernel_size_1: 3
filters_2: 32
kernel_size_2: 3
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.9886000156402588
Trial summary
Hyperparameters:
conv_layers: 3
dropout_rate: 0.2
activation: relu
optimizer: adam
learning_rate: 0.001
filters_0: 20
kernel_size_0: 5
filters_1: 28
kernel_size_1: 4
filters_2: 24
kernel_size_2: 4
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.9876000285148621
Trial summary
Hyperparameters:
conv_layers: 2
dropout_rate: 0.4
activation: relu
optimizer: adam
learning_rate: 0.0001
filters_0: 8
kernel_size_0: 5
filters_1: 28
kernel_size_1: 4
filters_2: 28
kernel_size_2: 5
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 0
tune

In [19]:
best_hps = hyperband_tuner.get_best_hyperparameters()[0]
print(best_hps.values)

best_model_hp = hyperband_tuner.get_best_models()[0]
best_model_hp.summary()

{'conv_layers': 2, 'dropout_rate': 0.0, 'activation': 'relu', 'optimizer': 'rmsprop', 'learning_rate': 0.001, 'filters_0': 20, 'kernel_size_0': 4, 'filters_1': 32, 'kernel_size_1': 5, 'filters_2': 20, 'kernel_size_2': 3, 'tuner/epochs': 2, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 20)        340       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        16032     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
flatten (Flatt

In [20]:
bo_tuner=BayesianOptimization(
    build_model,
    objective="val_accuracy",
    max_trials=15,
    num_initial_points=2,
    alpha=0.0001,
    beta=2.6,
    seed=0,
    hyperparameters=None,
    tune_new_entries=True,
    allow_new_entries=True,
    overwrite=True,
    directory=".",
    project_name="keras_trial",
   
)

In [21]:
bo_tuner.search(x_train, y_train, epochs=2, validation_data=(x_test, y_test))

Trial 15 Complete [00h 00m 16s]
val_accuracy: 0.9811000227928162

Best val_accuracy So Far: 0.9890999794006348
Total elapsed time: 00h 04m 10s
INFO:tensorflow:Oracle triggered exit


In [23]:
best_hps = bo_tuner.get_best_hyperparameters()[0]
print(best_hps.values)

best_model_hp = tuner.get_best_models()

{'conv_layers': 3, 'dropout_rate': 0.0, 'activation': 'relu', 'optimizer': 'adam', 'learning_rate': 0.0001, 'filters_0': 32, 'kernel_size_0': 5, 'filters_1': 32, 'kernel_size_1': 3, 'filters_2': 32, 'kernel_size_2': 3}


In [24]:
tuner.results_summary()

Results summary
Results in .r\mnist
Showing 10 best trials
Objective(name='val_accuracy', direction='max')


In [59]:
best_hps = bo_tuner.get_best_hyperparameters()[0]
print(best_hps.values)

best_model = hyperband_tuner.get_best_models()


{'conv_layers': 3, 'dropout_rate': 0.0, 'activation': 'relu', 'optimizer': 'adam', 'learning_rate': 0.0001, 'filters_0': 32, 'kernel_size_0': 5, 'filters_1': 32, 'kernel_size_1': 3, 'filters_2': 32, 'kernel_size_2': 3}


In [26]:
best_model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 20)        340       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        16032     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6272)              0         
_________________________________________________________________
dropout (Dropout)            (None, 6272)              0         
_________________________________________________________________
dense (Dense)                (None, 10)               

In [28]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = bo_tuner.hypermodel.build(best_hps)
history = model.fit(x_train, y_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.1517 - accuracy: 0.9529 - val_loss: 0.0565 - val_accuracy: 0.9831
Epoch 2/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.0526 - accuracy: 0.9842 - val_loss: 0.0546 - val_accuracy: 0.9838
Epoch 3/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.0372 - accuracy: 0.9885 - val_loss: 0.0524 - val_accuracy: 0.9859
Epoch 4/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.0297 - accuracy: 0.9909 - val_loss: 0.0512 - val_accuracy: 0.9856
Epoch 5/50
1500/1500 [==============================] - 5s 4ms/step - loss: 0.0222 - accuracy: 0.9926 - val_loss: 0.0564 - val_accuracy: 0.9840
Epoch 6/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.0174 - accuracy: 0.9941 - val_loss: 0.0543 - val_accuracy: 0.9866
Epoch 7/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.0136 - accuracy: 0.9952 - val_loss: 0.0571 - val_accuracy:

In [53]:
print(f"""
The hyperparameter search is complete.
The optimal number of conv layes is {best_hps.get('conv_layers')}
The optimal number of filters  in the first densely-connected layer is {best_hps.get('filters_0')} and 
The optimal number of filters  in the first densely-connected layer is {best_hps.get('filters_1')} 
The optimal learning rate for the optimizer is {best_hps.get('learning_rate')}. 
The optimal activation is {best_hps.get('activation')}
The best optimizer is {best_hps.get('optimizer')} 
Number of epochs with best val Accurcay of {max(val_acc_per_epoch)} is {best_epoch}
""")


The hyperparameter search is complete.
The optimal number of conv layes is 3
The optimal number of filters  in the first densely-connected layer is 32 and 
The optimal number of filters  in the first densely-connected layer is 32 
The optimal learning rate for the optimizer is 0.0001. 
The optimal activation is relu
The best optimizer is adam 
Number of epochs with best val Accurcay of 0.9903333187103271 is 15

